In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from rich.pretty import pprint
from torcheeg.datasets import DREAMERDataset
from torcheeg.datasets.constants.emotion_recognition.dreamer import DREAMER_CHANNEL_LOCATION_DICT
from torcheeg import transforms
from torch import nn
import torch
import random

In [2]:
dataset_path = "./DREAMER.mat"
base_path = "./"

toTensor = transforms.ToTensor()

In [20]:
dataset = DREAMERDataset(
    io_path=base_path + 'dreamer8sec_unnormalized',
    mat_path=dataset_path,
    offline_transform=transforms.Compose([
        transforms.BaselineRemoval(),
        # transforms.MeanStdNormalize(),
        transforms.To2d()
    ]),
    # online_transform=transforms.ToTensor(),
    label_transform=transforms.Compose(
        [transforms.Select('valence'),
         transforms.Binary(3.0)]),
    chunk_size=976,
    baseline_chunk_size=976,
    num_baseline=8
)

The target folder already exists, if you need to regenerate the database IO, please delete the path ./dreamer8sec_unnormalized.


In [21]:
def get_tf_feature(eeg, sr, n_channels = 14):
    WinLength = int(0.5*sr) # 500 points (0.5 sec, 500 ms)
    step = int(0.025*sr) # 25 points (or 25 ms)
    final_features = None
    for i in range(n_channels):
        eeg_single = eeg[i].squeeze()
        myparams = dict(nperseg = WinLength, noverlap = WinLength-step, return_onesided=True, mode='magnitude')
        f, nseg, Sxx = signal.spectrogram(x = eeg_single, fs = sr, **myparams)
        if(isinstance(final_features, np.ndarray)):
            final_features = np.concatenate((final_features, Sxx), axis=0)
        else:
            final_features = Sxx
    return final_features

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [74]:
abc = nn.Conv2d(1, 1, 3, stride=3)
abc1 = nn.Conv2d(1, 1, 5, stride=1)

In [75]:
dat = get_tf_feature(dataset[0][0][0], 128)
dat.shape

(462, 305)

In [76]:
dat = dat.astype("float32")
dat2 = dat.reshape(1, dat.shape[0], dat.shape[1])
d = torch.from_numpy(dat2)
c = abc(d)
c.shape

torch.Size([1, 154, 101])

In [77]:
abcd = nn.Flatten()

In [78]:
abc1(c).shape

torch.Size([1, 150, 97])

In [97]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2D_1 = nn.Sequential(
            nn.Conv2d(1, 1, 3, stride=3),
            nn.Conv2d(1, 1, 5, stride=1),
        )
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(14550, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )

    def forward(self, x):
        x = self.conv2D_1(x)
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [98]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (conv2D_1): Sequential(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(3, 3))
    (1): Conv2d(1, 1, kernel_size=(5, 5), stride=(1, 1))
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=14550, out_features=2048, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2048, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=2, bias=True)
  )
)


In [99]:
# X = torch.rand(1, 28, 28, device=device)
# logits = model(X)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")

In [100]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [101]:
def convert_data_to_tensor(data):
    data = data.astype("float32")
    data = data.reshape(1, dat.shape[0], dat.shape[1])
    return torch.from_numpy(data)

In [114]:
def train_loop(dataset,  model, loss_fn, optimizer):
    size = len(dataset)
    for i in range(size):
        # Compute prediction and loss
        X, y = dataset[i][0][0], dataset[i][1]
        X = get_tf_feature(X, sr=128)
        X = convert_data_to_tensor(X)
        if y == 0:
            y = [0]
        else:
            y = [1]
        y = torch.tensor(y)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            loss, current = loss.item(), i + 1
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [134]:
random.seed(42)

sample_size = 1000
l = random.sample(range(0, 11000), sample_size)
l

[10476,
 1824,
 409,
 4506,
 4012,
 3657,
 2286,
 1679,
 8935,
 1424,
 9674,
 6912,
 520,
 488,
 1535,
 3582,
 3811,
 8279,
 9863,
 434,
 9195,
 3257,
 10647,
 8928,
 6873,
 3611,
 7359,
 9654,
 4557,
 106,
 2615,
 6924,
 5574,
 4552,
 2547,
 3527,
 5514,
 1674,
 1519,
 6224,
 1584,
 5881,
 5635,
 9891,
 4333,
 711,
 7527,
 8785,
 2045,
 6201,
 1291,
 9044,
 4803,
 10299,
 10133,
 5925,
 9459,
 3150,
 1139,
 750,
 10834,
 3733,
 4741,
 1307,
 3814,
 1654,
 6227,
 4554,
 7428,
 10415,
 5977,
 2664,
 6065,
 5820,
 3432,
 10980,
 4374,
 10617,
 1169,
 9980,
 10403,
 2803,
 8751,
 4010,
 2677,
 7573,
 6216,
 4422,
 10485,
 9125,
 3598,
 5313,
 916,
 3752,
 525,
 5168,
 6572,
 4386,
 1084,
 3456,
 9292,
 5155,
 3483,
 10738,
 8179,
 6482,
 10532,
 7517,
 2340,
 4339,
 2287,
 4040,
 9197,
 8830,
 4304,
 9577,
 7019,
 9560,
 6543,
 5930,
 3593,
 2266,
 8348,
 8085,
 1489,
 771,
 1796,
 2504,
 10280,
 2621,
 6916,
 9771,
 1040,
 6304,
 6252,
 9763,
 7668,
 8669,
 4119,
 9064,
 188,
 1876,
 879

In [135]:
def test_loop(dataset, model, loss_fn):
    size = len(dataset)
    test_loss, correct = 0, 0
    sample_size = 1000
    # l = random.sample(range(0, 11000), 1)
    with torch.no_grad():
        for i in l:
            X, y = dataset[i][0][0], dataset[i][1]
            X = get_tf_feature(X, sr=128)
            X = convert_data_to_tensor(X)
            if y == 0:
                y = [0]
            else:
                y = [1]
            y = torch.tensor(y)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= sample_size
    correct /= sample_size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [136]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(dataset, model, loss_fn, optimizer)
    test_loop(dataset, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.015926  [    1/11040]
loss: 0.103227  [  101/11040]
loss: 0.000000  [  201/11040]
loss: 0.245797  [  301/11040]
loss: 0.592975  [  401/11040]
loss: 0.100824  [  501/11040]
loss: 0.023340  [  601/11040]
loss: 0.000293  [  701/11040]
loss: 0.097830  [  801/11040]
loss: 0.000253  [  901/11040]
loss: 0.081287  [ 1001/11040]
loss: 0.302080  [ 1101/11040]
loss: 0.176189  [ 1201/11040]
loss: 0.188418  [ 1301/11040]
loss: 1.205439  [ 1401/11040]
loss: 0.009578  [ 1501/11040]
loss: 0.016469  [ 1601/11040]
loss: 0.451299  [ 1701/11040]
loss: 0.775532  [ 1801/11040]
loss: 0.121244  [ 1901/11040]
loss: 0.048656  [ 2001/11040]
loss: 0.079172  [ 2101/11040]
loss: 0.410055  [ 2201/11040]
loss: 0.333061  [ 2301/11040]
loss: 0.396655  [ 2401/11040]
loss: 1.163746  [ 2501/11040]
loss: 1.085184  [ 2601/11040]
loss: 0.425549  [ 2701/11040]
loss: 0.404562  [ 2801/11040]
loss: 0.407916  [ 2901/11040]
loss: 0.408503  [ 3001/11040]
loss: 0.952876  [ 3101/11040]
